# Text Analytics | BAIS:6100
# Module 10: Text Similarity

Instructor: Kang-Pyo Lee 

Topics to be covered:
- Word similarity (+ exercises)
- Text similarity (+ exercises)

Text similarity is a number typically between 0 and 1 that tells us how close two pieces of text are. Text similarity has many applications. It can be used by search engines in modeling the relevance of a document to a search query and in expanding search terms with their similar terms. It can be used by recommendation engines that maintain lists of similar words or terms. It can be used to detect plagiarism between two documents.  

## Word Similarity

In [ ]:
word1 = "text"
word2 = "tests"

The Levenshtein distance, also known as edit distance, is a string metric for measuring the difference between two sequences. Informally, the Levenshtein distance between two words is the minimum number of single-character edits (insertions, deletions or substitutions) required to change one word into the other.

In [ ]:
from IPython.display import Image
Image("classdata/images/levenshtein_distance.png")

In [ ]:
import numpy as np

def get_levenshtein_dist(seq1, seq2):
    size_x = len(seq1) + 1
    size_y = len(seq2) + 1
    matrix = np.zeros ((size_x, size_y), dtype=int)
    for x in range(size_x):
        matrix [x, 0] = x
    for y in range(size_y):
        matrix [0, y] = y

    for x in range(1, size_x):
        for y in range(1, size_y):
            if seq1[x-1] == seq2[y-1]:
                matrix [x,y] = min(
                    matrix[x-1, y] + 1,
                    matrix[x-1, y-1],
                    matrix[x, y-1] + 1
                )
            else:
                matrix [x,y] = min(
                    matrix[x-1,y] + 1,
                    matrix[x-1,y-1] + 1,
                    matrix[x,y-1] + 1
                )
    
    return (matrix[size_x-1, size_y-1])

Levenshtein Distance and Text Similarity in Python: https://stackabuse.com/levenshtein-distance-and-text-similarity-in-python/

In [ ]:
get_levenshtein_dist("text", "text")

In [ ]:
get_levenshtein_dist("text", "texts")   # one addition

In [ ]:
get_levenshtein_dist("text", "test")    # one substitution 

In [ ]:
get_levenshtein_dist("text", "tests")   # one substitution + one addition 

In [ ]:
get_levenshtein_dist("text", "texture") # three additions  

In [ ]:
get_levenshtein_dist("text", "toast")   # two substitutions + one addition   

In [ ]:
hashtag = "covid19"

In [ ]:
N = 500

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', 150)

months = ["202012", "202011", "202010", "202009", "202008", "202007", 
          "202006", "202005", "202004", "202003", "202002", "202001"]

df = pd.DataFrame()
for month in months:
    dftmp = pd.read_csv("classdata/tweets/tweets_{}_{}.csv".format(hashtag, month), sep="\t", quoting=3)
    
    ##############################################
    # Create a random sample of N rows.
    ##############################################
    if len(dftmp) > N:
        dftmp = dftmp.sample(n=N)
    ##############################################
    
    df = pd.concat([df, dftmp])
    print("{}: {:,}".format(month, len(dftmp)))

print("Total number of tweets in df: {:,}\n".format(len(df)))

df.user_name = df.user_name.astype(str)
df.text = df.text.astype(str)
df = df.drop_duplicates(["text"])

df

In [ ]:
import nltk
import re

def get_unique_words(text_list):
    all_words = set()
    
    for text in text_list:
        words = nltk.word_tokenize(text)
        for word in words:
            if re.search("^[a-zA-Z][a-zA-Z0-9]+", word):  # Any word starting with an alphabet letter followed by any alphanumerical characters
                all_words.add(word.lower())
                
    return all_words

In [ ]:
unique_words = get_unique_words(df.text[:100])
unique_words

In [ ]:
unique_words = sorted(unique_words)     # A set becomes a list when sorted.
unique_words

In [ ]:
len(unique_words)

In [ ]:
for word1 in unique_words:
    if (len(word1) < 5) | (word1.endswith("…")):      # Filter out words that are too short or end with '…'
        continue
        
    for word2 in unique_words:
        if (len(word2) < 5) | (word2.endswith("…")):  # Filter out words that are too short or end with '…'
            continue
            
        if word1 != word2:
            dist = get_levenshtein_dist(word1, word2)
            
            if dist == 1:
                print(word1, word2)

The Levenshtein distance between any two words is symmetric. Two words with edit distance 1 are mostly word variations such as singular and plural nouns. Some of them are not similar at all such as bears and years. 

In [ ]:
stemmer = nltk.stem.SnowballStemmer("english")

for word1 in unique_words:
    if (len(word1) < 5) | (word1.endswith("…")):      # Filter out words that are too short or end with '…'
        continue
        
    for word2 in unique_words:
        if (len(word2) < 5) | (word2.endswith("…")):  # Filter out words that are too short or end with '…'
            continue
            
        if (word1 != word2) & (stemmer.stem(word1) == stemmer.stem(word2)):
            print(word1, word2)

## Exercises - Word Similarity

## Text Similarity

In [ ]:
text1 = "Hey, how are you?"
text2 = "How are you doing today?"

### Jaccard Similarity

Jaccard similarity is defined as size of intersection divided by size of union of two sets, in other words, intersection over union. 

In [ ]:
Image("classdata/images/sim_jaccard.png")

In [ ]:
import string

def tokenize(text):
    words = nltk.word_tokenize(text)
    words = [word.lower() for word in words if word not in string.punctuation]
    
    return words

In [ ]:
tokenize("Hey, how are you?")

In [ ]:
def get_jaccard_sim(text1, text2): 
    a = set(tokenize(text1)) 
    b = set(tokenize(text2))
    i = a & b
    u = a | b
    
    return len(i) / len(u)

In [ ]:
get_jaccard_sim(text1, text2)

### Cosine Similarity

Cosine similarity calculates similarity by measuring the cosine of angle between two vectors. The closer the angle of two vectors, the higher the cosine similarity.  

In [ ]:
Image(url="https://d3i71xaburhd42.cloudfront.net/d203f5734f5ee9d49c0adff31805ed93034ca60e/3-Figure1-1.png")

In [ ]:
Image(url="https://neo4j.com/docs/graph-algorithms/current/images/cosine-similarity.png")

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def get_cosine_sim(text1, text2):
    corpus = [text1, text2]
    vectorizer = TfidfVectorizer(use_idf=False, norm=None)
    dtm = vectorizer.fit_transform(corpus)
    
    return cosine_similarity(dtm)[0][1]

sklearn.metrics.pairwise.cosine_similarity: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html

With cosine similarity, we need to convert sentences into vectors. The choice of TF or TF-IDF depends on application and is immaterial to how cosine similarity is actually performed — which just needs vectors. TF is good for text similarity in general, but TF-IDF is good for search query relevance.

The <b>cosine_similarity</b> module returns an array of the pairwise similarties between all samples.

In [ ]:
df

In [ ]:
df.index = range(len(df))
df

In [ ]:
from nltk.corpus import stopwords

global_stopwords = stopwords.words("english")
local_stopwords = [c for c in string.punctuation] +\
                  ['’', '``', '…', '...', "''", '‘', '“', '”', "'m", "'re", "'s", "'ve", 'amp', 'https', "n't", 'rt', 
                   'covid19', 'coronavirus', 'covid19…', 'covid', 'co']

vectorizer = TfidfVectorizer(use_idf=True, norm="l2", stop_words=global_stopwords+local_stopwords, max_df=0.7)
dtm = vectorizer.fit_transform(df.text)

In [ ]:
cosine_similarity(dtm)

In [ ]:
cosine_similarity(dtm).shape

In [ ]:
df_sim = pd.DataFrame(data=cosine_similarity(dtm),
                      columns=df.index, index=df.index)
df_sim

All values on the diagonal are 1.0 as the similarity of something to itself is always 1. 

In [ ]:
for pos1 in df_sim.index:
    for pos2 in df_sim.index:
        if pos1 != pos2:
            text1 = df.text[pos1]
            text2 = df.text[pos2]
            
            # Skip if the first 20 characters of one text are in the other text
            if (text1[:20] in text2) | (text2[:20] in text1):
                continue
            
            sim = df_sim[pos2][pos1]
            
            if (sim > 0.7) & (sim < 1):
                print(text1)
                print(text2)
                print("{:.3f}".format(sim))
                print()

Differences between Jaccard Similarity and Cosine Similarity:
- Jaccard similarity takes only unique set of words for each sentence/document while cosine similarity takes total length of the vectors. 
- Jaccard similarity is good for cases where duplication does not matter, cosine similarity is good for cases where duplication matters while analyzing text similarity. For two product descriptions, it will be better to use Jaccard similarity as repetition of a word does not reduce their similarity.

Overview of Text Similarity Metrics in Python: https://towardsdatascience.com/overview-of-text-similarity-metrics-3397c4601f50

## Exercises - Text Similarity